In [7]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages  # Import PdfPages for saving to a PDF
import torch

# Import the predict_img function and models from the previous script
from unet import UNet
from utils import hwc_to_chw, h5_utils as h5u

# Path to the folder containing the output files
output_folder = '/exp/sbnd/data/users/abhat/wirecell_data/dnn_roi/ICARUS/81858535_63/'  # Change this to your output folder
pdf_filename = "81858535_63_images.pdf"  # Name of the output PDF should be similar to the last folder path above

# Define TPCs and planes
tpcs=[0,1,2,3]
planes = ['front_induction', 'middle_induction']
plane_suffix_map = {'front_induction': '0', 'middle_induction': '1'}

# Specify which event to plot (index 0 to 9)
event_to_plot = 9  # Change this to the event index you want to plot

# Load U and V Plane models
model_u_path = '/scratch/7DayLifetime/abhat/wirecell/dnn_roi_icarus/training/bnb_cosmics/UNet/U_Plane/best_loss.pth'
model_v_path = '/scratch/7DayLifetime/abhat/wirecell/dnn_roi_icarus/training/bnb_cosmics/UNet/V_Plane/best_loss.pth'

net_u = UNet(2, 1)
net_u.load_state_dict(torch.load(model_u_path, map_location='cpu'))
net_u.eval()

net_v = UNet(2, 1)
net_v.load_state_dict(torch.load(model_v_path, map_location='cpu'))
net_v.eval()


def predict_img(net, full_img, scale_factor=0.5, out_threshold=0.5, use_gpu=False):
    img_tensor = torch.from_numpy(hwc_to_chw(full_img))
    if use_gpu:
        img_tensor = img_tensor.cuda()

    with torch.no_grad():
        input = img_tensor.unsqueeze(0)
        full_mask = net(input).squeeze().cpu().numpy()

    if out_threshold < 0:
        return full_mask

    return full_mask > out_threshold

def model_selection(file_path, plane):
    if plane == "U":
        net = net_u
        img = h5u.get_hwc_img(file_path, event_to_plot, ['frame_looseLf', 'frame_mp3ROI'], [1, 8], [0, 2112], [0, 4096], 2000)
    elif plane == "V":
        net = net_v
        img = h5u.get_hwc_img(file_path, event_to_plot, ['frame_looseLf', 'frame_mp3ROI'], [1, 8], [0, 5600], [0, 4096], 2000)

    # Make the prediction
    predicted_mask = predict_img(net=net, full_img=img, scale_factor=0.5, out_threshold=0.5, use_gpu=False)
    return np.transpose(predicted_mask)

# Define the function to plot the data for each TPC and plane
def plot_data(tpc, plane, plane_suffix, data_dict, event_idx, predicted_image, pdf):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))  # Adjust grid to fit only 4 images (DepoSplat, LooseLF, MP3 -> MP2, Predicted)
    titles = ["DepoSplat", "LooseLF", "MP3ROI", "Predicted"]

    # Plot DepoSplat, LooseLF, and MP3 as MP2
    for i, (key, data) in enumerate(data_dict.items()):
        if np.any(data) and i < 2:  # Only plot DepoSplat and LooseLF here
            ax = axes[i//2, i%2]
            ax.imshow(data, aspect='auto', cmap="bwr", vmin=-1000, vmax=1000, origin='lower')
            ax.set_title(f"{titles[i]} TPC{tpc} Plane{plane_suffix} (Event {event_idx})")
            ax.set_xlabel("Channels")
            ax.set_ylabel("Ticks")

    # Plot MP3 image in place of MP2
    ax = axes[1, 0]
    ax.imshow(data_dict["MP3ROI"], aspect='auto', cmap="bwr", vmin=-1000, vmax=1000, origin='lower')
    ax.set_title(f"MP3ROI TPC{tpc} Plane{plane_suffix} (Event {event_idx})")
    ax.set_xlabel("Channels")
    ax.set_ylabel("Ticks")

    # Plot the predicted image
    ax = axes[1, 1]  # Place in the next available subplot
    ax.imshow(predicted_image, aspect='auto', cmap="bwr", origin='lower')
    ax.set_title(f"Predicted Image TPC{tpc} Plane{plane_suffix} (Event {event_idx})")
    ax.set_xlabel("Channels")
    ax.set_ylabel("Ticks")

    plt.tight_layout()
    pdf.savefig(fig)  # Save the figure to the PDF
    plt.close()  # Close the figure after saving to avoid displaying it inline

# Inspect each file for each TPC and plane and save all images to a single PDF
with PdfPages(pdf_filename) as pdf:  # Open the PDF file to save the images
    for tpc in tpcs:
        for plane, plane_suffix in plane_suffix_map.items():
            # Construct file paths
            true_file_path = os.path.join(output_folder, f"tpc{tpc}_plane{plane_suffix}_tru.h5")
            rec_file_path = os.path.join(output_folder, f"tpc{tpc}_plane{plane_suffix}_rec.h5")

            if os.path.exists(true_file_path) and os.path.exists(rec_file_path):
                print(f"Opening files for TPC {tpc}, Plane {plane}:")
                print(f"True file: {true_file_path}")
                print(f"Reconstructed file: {rec_file_path}")

                # Open the files
                with h5py.File(true_file_path, 'r') as true_file, h5py.File(rec_file_path, 'r') as rec_file:
                    # Ensure the event_to_plot exists in the file
                    events = list(true_file.keys())
                    if 0 <= event_to_plot < len(events):
                        event = events[event_to_plot]
                        print(f"Processing event {event} for TPC {tpc}, Plane {plane}...")

                        # Load data for the specified event
                        true_data = true_file[event]['frame_deposplat'][:]
                        rec_data_looseLf = rec_file[event]['frame_looseLf'][:]
                        rec_data_mp3 = rec_file[event]['frame_mp3ROI'][:]  # Using MP3 in place of MP2

                        # Prepare data to plot (no MP2, MP3 in place)
                        data_dict = {
                            "DepoSplat": true_data,
                            "LooseLF": rec_data_looseLf,
                            "MP3ROI": rec_data_mp3
                        }

                        # Generate the predicted image
                        predicted_image = model_selection(rec_file_path, "U" if plane_suffix == "0" else "V")

                        # Plot the data for the specified event and predicted image, and save to the PDF
                        plot_data(tpc, plane, plane_suffix, data_dict, event_to_plot, predicted_image, pdf)
                    else:
                        print(f"Event index {event_to_plot} is out of range for TPC {tpc}, Plane {plane}.")

            else:
                print(f"Files for TPC {tpc}, Plane {plane} not found.")

print(f"All images have been saved to {pdf_filename}.")


Opening files for TPC 0, Plane front_induction:
True file: /exp/sbnd/data/users/abhat/wirecell_data/dnn_roi/ICARUS/81858535_63/tpc0_plane0_tru.h5
Reconstructed file: /exp/sbnd/data/users/abhat/wirecell_data/dnn_roi/ICARUS/81858535_63/tpc0_plane0_rec.h5
Processing event 9 for TPC 0, Plane front_induction...
Opening files for TPC 0, Plane middle_induction:
True file: /exp/sbnd/data/users/abhat/wirecell_data/dnn_roi/ICARUS/81858535_63/tpc0_plane1_tru.h5
Reconstructed file: /exp/sbnd/data/users/abhat/wirecell_data/dnn_roi/ICARUS/81858535_63/tpc0_plane1_rec.h5
Processing event 9 for TPC 0, Plane middle_induction...
Opening files for TPC 1, Plane front_induction:
True file: /exp/sbnd/data/users/abhat/wirecell_data/dnn_roi/ICARUS/81858535_63/tpc1_plane0_tru.h5
Reconstructed file: /exp/sbnd/data/users/abhat/wirecell_data/dnn_roi/ICARUS/81858535_63/tpc1_plane0_rec.h5
Processing event 9 for TPC 1, Plane front_induction...
Opening files for TPC 1, Plane middle_induction:
True file: /exp/sbnd/data

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages  # Import PdfPages for saving to a PDF
import torch

# Import the predict_img function and models from the previous script
from unet import UNet
from utils import hwc_to_chw, h5_utils as h5u

# Path to the main folder containing subdirectories
main_folder = '/exp/sbnd/data/users/abhat/wirecell_data/dnn_roi/ICARUS/'

# Define TPCs and planes
tpcs = [0, 1, 2, 3]
planes = ['front_induction', 'middle_induction']
plane_suffix_map = {'front_induction': '0', 'middle_induction': '1'}

# Specify which event to plot (index 0 to 9)
event_to_plot = 9  # Change this to the event index you want to plot

# Load U and V Plane models
model_u_path = '/scratch/7DayLifetime/abhat/wirecell/dnn_roi_icarus/training/bnb_cosmics/UNet/U_Plane/best_loss.pth'
model_v_path = '/scratch/7DayLifetime/abhat/wirecell/dnn_roi_icarus/training/bnb_cosmics/UNet/V_Plane/best_loss.pth'

net_u = UNet(2, 1)
net_u.load_state_dict(torch.load(model_u_path, map_location='cpu'))
net_u.eval()

net_v = UNet(2, 1)
net_v.load_state_dict(torch.load(model_v_path, map_location='cpu'))
net_v.eval()

def predict_img(net, full_img, scale_factor=0.5, out_threshold=0.5, use_gpu=False):
    img_tensor = torch.from_numpy(hwc_to_chw(full_img))
    if use_gpu:
        img_tensor = img_tensor.cuda()

    with torch.no_grad():
        input = img_tensor.unsqueeze(0)
        full_mask = net(input).squeeze().cpu().numpy()

    if out_threshold < 0:
        return full_mask

    return full_mask > out_threshold

def model_selection(file_path, plane):
    if plane == "U":
        net = net_u
        img = h5u.get_hwc_img(file_path, event_to_plot, ['frame_looseLf', 'frame_mp3ROI'], [1, 8], [0, 2112], [0, 4096], 2000)
    elif plane == "V":
        net = net_v
        img = h5u.get_hwc_img(file_path, event_to_plot, ['frame_looseLf', 'frame_mp3ROI'], [1, 8], [0, 5600], [0, 4096], 2000)

    predicted_mask = predict_img(net=net, full_img=img, scale_factor=0.5, out_threshold=0.5, use_gpu=False)
    return np.transpose(predicted_mask)

def plot_data(tpc, plane, plane_suffix, data_dict, event_idx, predicted_image, pdf):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    titles = ["DepoSplat", "LooseLF", "MP3ROI", "Predicted"]

    for i, (key, data) in enumerate(data_dict.items()):
        if np.any(data) and i < 2:
            ax = axes[i//2, i%2]
            ax.imshow(data, aspect='auto', cmap="bwr", vmin=-1000, vmax=1000, origin='lower')
            ax.set_title(f"{titles[i]} TPC{tpc} Plane{plane_suffix} (Event {event_idx})")
            ax.set_xlabel("Channels")
            ax.set_ylabel("Ticks")

    ax = axes[1, 0]
    ax.imshow(data_dict["MP3ROI"], aspect='auto', cmap="bwr", vmin=-1000, vmax=1000, origin='lower')
    ax.set_title(f"MP3ROI TPC{tpc} Plane{plane_suffix} (Event {event_idx})")
    ax.set_xlabel("Channels")
    ax.set_ylabel("Ticks")

    ax = axes[1, 1]
    ax.imshow(predicted_image, aspect='auto', cmap="bwr", origin='lower')
    ax.set_title(f"Predicted Image TPC{tpc} Plane{plane_suffix} (Event {event_idx})")
    ax.set_xlabel("Channels")
    ax.set_ylabel("Ticks")

    plt.tight_layout()
    pdf.savefig(fig)
    plt.close()

# Iterate through all subdirectories in the main folder
with PdfPages("ICARUS_images.pdf") as pdf:
    for folder_name in os.listdir(main_folder):
        folder_path = os.path.join(main_folder, folder_name)
        if os.path.isdir(folder_path):  # Ensure it's a directory
            for tpc in tpcs:
                for plane, plane_suffix in plane_suffix_map.items():
                    true_file_path = os.path.join(folder_path, f"tpc{tpc}_plane{plane_suffix}_tru.h5")
                    rec_file_path = os.path.join(folder_path, f"tpc{tpc}_plane{plane_suffix}_rec.h5")

                    if os.path.exists(true_file_path) and os.path.exists(rec_file_path):
                        print(f"Processing TPC {tpc}, Plane {plane} in folder {folder_name}")

                        with h5py.File(true_file_path, 'r') as true_file, h5py.File(rec_file_path, 'r') as rec_file:
                            events = list(true_file.keys())
                            if 0 <= event_to_plot < len(events):
                                event = events[event_to_plot]

                                true_data = true_file[event]['frame_deposplat'][:]
                                rec_data_looseLf = rec_file[event]['frame_looseLf'][:]
                                rec_data_mp3 = rec_file[event]['frame_mp3ROI'][:]

                                data_dict = {
                                    "DepoSplat": true_data,
                                    "LooseLF": rec_data_looseLf,
                                    "MP3ROI": rec_data_mp3
                                }

                                predicted_image = model_selection(rec_file_path, "U" if plane_suffix == "0" else "V")

                                plot_data(tpc, plane, plane_suffix, data_dict, event_to_plot, predicted_image, pdf)
                            else:
                                print(f"Event index {event_to_plot} is out of range for TPC {tpc}, Plane {plane}.")
                    else:
                        print(f"Files for TPC {tpc}, Plane {plane} not found in folder {folder_name}.")

print("All images have been saved to a single PDF.")


Processing TPC 0, Plane front_induction in folder 81858535_74


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


Processing TPC 0, Plane middle_induction in folder 81858535_74
Processing TPC 1, Plane front_induction in folder 81858535_74
Processing TPC 1, Plane middle_induction in folder 81858535_74
Processing TPC 2, Plane front_induction in folder 81858535_74
Processing TPC 2, Plane middle_induction in folder 81858535_74
Processing TPC 3, Plane front_induction in folder 81858535_74
Processing TPC 3, Plane middle_induction in folder 81858535_74
Processing TPC 0, Plane front_induction in folder 81858535_72
Processing TPC 0, Plane middle_induction in folder 81858535_72
Processing TPC 1, Plane front_induction in folder 81858535_72
Processing TPC 1, Plane middle_induction in folder 81858535_72
Processing TPC 2, Plane front_induction in folder 81858535_72
Processing TPC 2, Plane middle_induction in folder 81858535_72
Processing TPC 3, Plane front_induction in folder 81858535_72
Processing TPC 3, Plane middle_induction in folder 81858535_72
Processing TPC 0, Plane front_induction in folder 81858535_54


KeyboardInterrupt: 